In [3]:
binary_int = int("11000010110001001100011", 2);
binary_array = binary_int.to_bytes(3, "big")
ascii_text = binary_array.decode()
print(ascii_text)

abc


In [112]:
import numpy as np
import string

In [143]:
def init_populations(chrom_length,popu_length,alphabet):
    individuals = []
    for i in range(popu_length):
        chromosome = []
        for j in range(chrom_length):
            p = np.random.randint(0,len(alphabet))
            chromosome.append(alphabet[p])
        individuals.append(chromosome)
    return individuals

In [144]:
alphabet=string.ascii_letters+" ,."
ch1=init_populations(32,10,alphabet)
ch1[0]

[',',
 'p',
 'h',
 'W',
 'E',
 ',',
 'H',
 'y',
 'K',
 'C',
 'H',
 'S',
 '.',
 'i',
 '.',
 'F',
 'j',
 'a',
 'o',
 'f',
 'w',
 'h',
 'w',
 'q',
 'V',
 '.',
 'A',
 'h',
 'O',
 'g',
 'Y',
 'd']

In [145]:
# def decode(chromosome):
#     decoded=[]
#     for letter in chromosome:
#         binary_array = letter.to_bytes(1, "big")
#         ascii_text = binary_array.decode()
#         decoded.append(ascii_text)
#     decoded="".join(decoded)
#     return decoded

In [146]:
#decode(ch1[0])

In [147]:
def fitness(chromosome,target):
    # decoded_chromosome=decode(chromosome)
    count=0
    for i in range(len(target)):
        # if decoded_chromosome[i]==target[i]:
        if chromosome[i]==target[i]:
            count+=1
    
    return count

In [148]:
target="Me llamo Juan y mi compa Manuel."
fitness(ch1[0],target)

0

In [149]:
def prob_fitness(pop,target):
    probs = []
    cumulative_prob = []
    fitnesses = []
    acc = 0

    for i in range(len(pop)):
        fitnesses.append( fitness(pop[i],target) )
        acc += fitnesses[i]
        cumulative_prob.append(acc)

    for j in range(len(pop)):
        if acc!=0:
            probs.append(fitnesses[j]/acc)
        else:
            probs.append(0)

    return probs, cumulative_prob,fitnesses

In [326]:
prob_fitness(["pola","foca",target,target,target],target)

([0.17647058823529413,
  0.11764705882352941,
  0.23529411764705882,
  0.23529411764705882,
  0.23529411764705882],
 [3, 5, 9, 13, 17],
 [3, 2, 4, 4, 4])

In [252]:
def roullete(cumulative_fit,pop):
    selected_ind = []

    for i in range(2):
        ran = np.random.rand() * cumulative_fit[-1]

        s = 0

        while ran > cumulative_fit[s]:
            s += 1

        selected_ind.append(pop[s])
    return selected_ind

In [311]:
def select_by_fitness(fitness,pop,parent_size):
    sorted_ind=sorted(range(len(fitness)), key=fitness.__getitem__,reverse=True)
    # sorted_ind=np.argsort(fitness)
    sorted_pop=[]
    for i in range(parent_size):
        sorted_pop.append(pop[sorted_ind[i]])
    # sorted_pop=pop[sorted_ind]
    # return sorted_pop[:parent_size]
    return sorted_pop

In [309]:
def crossover(p_cross, cumulative_fit, popu_length,pop,chrom_length,fit):
    
    new_gen = []
    parents=select_by_fitness(fit,pop,popu_length//2)
    diff_parents=False

    while len(new_gen)<popu_length:
        
        # selected_ind = roullete(cumulative_fit,pop)
        # ind1 = selected_ind[0]
        # ind2 = selected_ind[1]
        while not diff_parents:
            x1=np.random.randint(0,len(parents))
            x2=np.random.randint(0,len(parents))
            if x1!=x2:
                diff_parents=True
        ind1=parents[x1]
        ind2=parents[x2]
            

        p = np.random.rand()

        if p < p_cross:

            cut = np.random.randint(1,chrom_length-1)
            # print(cut)
            new_ind1 = ind1[:cut] + ind2[cut:]
            new_ind2 = ind2[:cut] + ind1[cut:]
            # print("new_ind1",decode(new_ind1))
            # print("new_ind2",decode(new_ind2))
            new_gen.append(new_ind1)
            new_gen.append(new_ind2)
            # print("\n")
    return new_gen

In [254]:
p_cross , cumulative_fit , popu_length , chrom_length = 0.9 , prob_fitness(ch1,target)[1] , 10 , 32
new_gen=crossover(p_cross, cumulative_fit, popu_length,ch1,chrom_length)

In [314]:
def mutation(new_gen,p_mutation,chrom_length,alphabet):
    for i in range(len(new_gen)):
        # print("before",new_gen[i])
        for j in range(chrom_length):
            p= np.random.rand()
            if p<p_mutation:
                letter=np.random.randint(0,len(alphabet))
                new_gen[i][j]=alphabet[letter]
                # letter=np.random.randint(32,126)
                # if new_gen[i][j]!=letter:
                #     new_gen[i][j]=letter
                # else:
                    # # while new_gen[i][j]!=letter:
                    # while new_gen[i][j]!=alphabet[letter]:
                    #     letter=np.random.randint(0,len(alphabet))
                    #     new_gen[i][j]=alphabet[letter]
                    #     # letter=np.random.randint(32,126)
                    #     # new_gen[i][j]=letter
        # print("after",new_gen[i])
    return new_gen

In [279]:
new_mutated=mutation(new_gen,0.04,chrom_length,alphabet)

In [331]:
def genetic_algo(target,alphabet,p_cross,p_mutation,popu_length,chrom_length,number_gen):#,random_state):
    # np.random.seed(random_state)
    
    min, max, mean, best_chrom = [], [],[], []

    pop = init_populations(chrom_length,popu_length,alphabet)
    probs, cumulative_fit, fitnesses = prob_fitness(pop,target)
    print("fitnesses gen 0:",fitnesses)
    best_chrom.append(pop[np.argmax(fitnesses)])
    
    min.append(np.min(fitnesses))
    max.append(np.max(fitnesses))
    mean.append(np.mean(fitnesses))
    
    for i in range(number_gen):
        new_gen = crossover(p_cross, cumulative_fit, popu_length,pop,chrom_length,fitnesses)
        new_gen = mutation(new_gen,p_mutation,chrom_length,alphabet)
        probs, cumulative_fit, fitnesses = prob_fitness(new_gen,target)

        best_chrom.append(new_gen[np.argmax(fitnesses)])
        # if (i+1)%50==0:
        #     print(f"Generation {i+1}: {best_chrom[-1]}")
        print(f"Generation {i+1}: {new_gen[np.argmax(fitnesses)]}, fitness:{fitnesses[np.argmax(fitnesses)]}")
        min.append(np.min(fitnesses))
        max.append(np.max(fitnesses))
        mean.append(np.mean(fitnesses))

    return min, max, mean, best_chrom

In [332]:
p_cross=0.8
popu_length=100
p_mutation=0.05
number_gen=1000
rand_state=42
target="hola"
chrom_length=len(target)
# alphabet=string.ascii_letters+" ,."
alphabet=string.ascii_lowercase

min, max, mean, best_chrom = genetic_algo(target,alphabet,p_cross,p_mutation,popu_length,chrom_length,number_gen)#,rand_state)
best_chrom

fitnesses gen 0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
Generation 1: ['v', 'v', 'a', 'w'], fitness:0
Generation 2: ['s', 'm', 'w', 'z'], fitness:0
Generation 3: ['m', 'n', 'x', 'y'], fitness:0
Generation 4: ['q', 'q', 'h', 'l'], fitness:0
Generation 5: ['n', 'o', 'b', 'e'], fitness:1
Generation 6: ['o', 'n', 'w', 'z'], fitness:0
Generation 7: ['i', 'i', 'q', 't'], fitness:0
Generation 8: ['i', 'q', 'l', 'l'], fitness:1
Generation 9: ['q', 'f', 'b', 'a'], fitness:1
Generation 10: ['v', 'k', 'x', 'a'], fitness:1
Generation 11: ['h', 'c', 'p', 'q'], fitness:1
Generation 12: ['o', 'o', 'z', 'h'], fitness:1
Generation 13: ['p', 'x', 'e', 'a'], fitness:1
Generation 14: ['l', 'i', 'j', 'j'], fitness:0
Generation 15: ['f', 'l', 'g', 'v

[['y', 'o', 't', 'a'],
 ['v', 'v', 'a', 'w'],
 ['s', 'm', 'w', 'z'],
 ['m', 'n', 'x', 'y'],
 ['q', 'q', 'h', 'l'],
 ['n', 'o', 'b', 'e'],
 ['o', 'n', 'w', 'z'],
 ['i', 'i', 'q', 't'],
 ['i', 'q', 'l', 'l'],
 ['q', 'f', 'b', 'a'],
 ['v', 'k', 'x', 'a'],
 ['h', 'c', 'p', 'q'],
 ['o', 'o', 'z', 'h'],
 ['p', 'x', 'e', 'a'],
 ['l', 'i', 'j', 'j'],
 ['f', 'l', 'g', 'v'],
 ['b', 'w', 'l', 'k'],
 ['r', 'f', 'o', 'm'],
 ['i', 'o', 'j', 'y'],
 ['w', 's', 'l', 'z'],
 ['a', 'h', 'p', 't'],
 ['p', 'o', 'm', 'r'],
 ['p', 'o', 'h', 'i'],
 ['s', 'o', 'm', 'r'],
 ['h', 'p', 'b', 'n'],
 ['d', 'o', 'z', 'i'],
 ['r', 'p', 'y', 'o'],
 ['s', 'y', 'q', 'e'],
 ['b', 'h', 'k', 'e'],
 ['h', 'h', 'l', 'k'],
 ['u', 'v', 'a', 'a'],
 ['p', 'h', 'g', 'm'],
 ['p', 'y', 'l', 'w'],
 ['m', 'y', 'g', 'v'],
 ['p', 'h', 'y', 'q'],
 ['h', 'y', 's', 'i'],
 ['r', 'h', 'n', 'a'],
 ['q', 'p', 's', 'k'],
 ['k', 'f', 'g', 'm'],
 ['a', 'n', 'j', 'a'],
 ['q', 'd', 'l', 'e'],
 ['r', 'z', 'l', 'd'],
 ['h', 'z', 'l', 'z'],
 ['h', 's',